In [1]:
import boto3
import os

aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']
aws_region = os.environ['AWS_REGION']

client = boto3.client('lambda', region_name=aws_region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [3]:
fs = client.list_functions()['Functions']

In [4]:
len(fs)

2

### Inspecting roles and policies

In [ ]:
role_arn = role_response['Role']['Arn']
role = iam_client.get_role(RoleName=role_name)['Role']
pprint(role)

{'Arn': 'arn:aws:iam::723085979813:role/HelloWorldLambdaRole',
 'AssumeRolePolicyDocument': {'Statement': [{'Action': 'sts:AssumeRole',
                                             'Effect': 'Allow',
                                             'Principal': {'Service': 'lambda.amazonaws.com'}}],
                              'Version': '2012-10-17'},
 'CreateDate': datetime.datetime(2023, 12, 3, 11, 6, 13, tzinfo=tzutc()),
 'MaxSessionDuration': 3600,
 'Path': '/',
 'RoleId': 'AROA2QW2IOCSYACZEWHBW',
 'RoleLastUsed': {'LastUsedDate': datetime.datetime(2023, 12, 3, 11, 14, 14, tzinfo=tzutc()),
                  'Region': 'us-east-1'},
 'RoleName': 'HelloWorldLambdaRole'}


In [ ]:
policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
pprint(policies)

[{'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole',
  'PolicyName': 'AWSLambdaBasicExecutionRole'}]


In [ ]:
policy = iam_client.get_policy(PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')['Policy']
pprint(policy)

{'Arn': 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole',
 'AttachmentCount': 1,
 'CreateDate': datetime.datetime(2015, 4, 9, 15, 3, 43, tzinfo=tzutc()),
 'DefaultVersionId': 'v1',
 'Description': 'Provides write permissions to CloudWatch Logs.',
 'IsAttachable': True,
 'Path': '/service-role/',
 'PermissionsBoundaryUsageCount': 0,
 'PolicyId': 'ANPAJNCQGXC42545SKXIK',
 'PolicyName': 'AWSLambdaBasicExecutionRole',
 'Tags': [],
 'UpdateDate': datetime.datetime(2015, 4, 9, 15, 3, 43, tzinfo=tzutc())}


### Delete all resources

In [ ]:
for d in scheduler_client.list_schedules()['Schedules']:
    scheduler_client.delete_schedule(Name=d['Name'])

for d in lambda_client.list_functions()['Functions']:
    lambda_client.delete_function(FunctionName=d['FunctionName'])

for d in iam_client.list_policies(Scope='Local')['Policies']:
    iam_client.delete_policy(PolicyArn=d['Arn'])
    print(f'Deleted: {d["PolicyName"]}')
    
for d in iam_client.list_roles()['Roles']:
    try:
        iam_client.delete_role(RoleName=d['RoleName'])
        print(f'Deleted: {d["RoleName"]}')
    except ClientError as e:
        print(f'Cannot delete: ' + d['RoleName'])